Import the necessary modules.

In [ ]:
import state_est
from pandapower.networks import simple_four_bus_system

Run using `simple_four_bus_system()`.

In [ ]:
state_est.main()

===running <function case9 at 0x00000193FA38D0D0> net===
===running <function case9 at 0x00000193FA38D0D0> net===
Bad data found
Voltages: 
0    1.000000
1    1.000000
2    1.000000
3    0.987007
4    0.975472
5    1.003375
6    0.985645
7    0.996185
8    0.957621
Name: vm_pu, dtype: float64

 Voltage angles:  0    0.000000
1    9.668741
2    4.771073
3   -2.406644
4   -4.017264
5    1.925602
6    0.621545
7    3.799120
8   -4.349934
Name: va_degree, dtype: float64
      vm_pu  va_degree          p_mw        q_mvar
0  1.000000   0.000000 -7.195470e+01 -2.406896e+01
1  1.000000   9.668741 -1.630000e+02 -1.446012e+01
2  1.000000   4.771073 -8.500000e+01  3.649026e+00
3  0.987007  -2.406644  8.321842e-09 -7.044409e-09
4  0.975472  -4.017264  9.000000e+01  3.000000e+01
5  1.003375   1.925602 -1.713335e-09  6.666733e-09
6  0.985645   0.621545  1.000000e+02  3.500000e+01
7  0.996185   3.799120 -7.084074e-10 -1.149656e-10
8  0.957621  -4.349934  1.250000e+02  5.000000e+01
      vm_pu  va_deg